In [209]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from scipy.optimize import minimize

In [210]:
df_1= pd.read_excel('E:\\ANTAM\\DATA\\Merge Table to_RK and to_EF part 2.xlsx', sheet_name='shifting_7-7_16', skiprows=0)
# df_1 = df_1.rename(columns={
#     'Unnamed: 0': 'date',
#     'Unnamed: 1': 'time',
#     # tambahkan kolom lain sesuai kebutuhan
# })
df_1

,date,time,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2021-01-01,00:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01,01:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01,02:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01,03:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01,04:00:00,2.090,12.680,37.850,0.420,20.130,1.25,6.060000,1.880000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,2021-01-31,19:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,612.0,-3.3,0.55
603,2021-01-31,20:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,615.0,-5.0,0.55
604,2021-01-31,21:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,620.0,-2.4,0.54
605,2021-01-31,22:00:00,2.115,14.555,37.005,0.575,19.845,2.77,6.912904,1.864503,...,17.99,1.483,1.757,76.737,0.4162,0.055,5.651667,617.0,-3.2,0.54


In [211]:
df = df_1.drop(columns=['date', 'time']).copy()
df.columns
df.head()

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.09,12.68,37.85,0.42,20.13,1.25,6.06,1.88,0.54,11.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
df = df.drop(index=range(0, 16)).reset_index(drop=True)
df = df.iloc[:-7].reset_index(drop=True)
input_cols = df.loc[:, 'ni_in':'t_tic172'].columns
output_cols = df.loc[:, 'furnace_temp':'loi_kalsin'].columns

In [213]:
df[input_cols.union(output_cols)] = df[input_cols.union(output_cols)].interpolate(method='linear', axis=0)
print(df[input_cols.union(output_cols)].isnull().sum())

a_f_ratio           0
al2o3_in            0
bc                  0
c_met               0
cao_in              0
charge_kiln         0
current             0
fc_coal             0
fe_in               0
fe_met              0
fe_ni               0
fe_slag             0
furnace_temp        0
gcv_coal            0
kg_tco              0
load                0
loi_in              0
loi_kalsin          0
mc_kilnfeed         0
mgo_in              0
ni_in               0
ni_met              0
ni_slag             0
pic_161             0
pry_p               0
pry_v               0
reductor_consume    0
reductor_ratio      0
rpm                 0
s_m                 0
s_met               0
sec_p               0
sec_v               0
si_met              0
sio2_in             0
t_kalsin            0
t_tic162            0
t_tic163            0
t_tic166            0
t_tic172            0
tco                 0
total_coal          0
voltage             0
dtype: int64


In [214]:
# X_merged = pd.concat([normalized_df[input_cols], pca_df], axis=1)
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_norm = pd.DataFrame(scaler_x.fit_transform(df[input_cols]), columns=input_cols)
y_norm = pd.DataFrame(scaler_y.fit_transform(df[output_cols]), columns=output_cols)
# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.2, random_state=42)
X_train

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
77,0.873684,0.566532,0.695225,0.349515,0.414134,0.560510,0.358249,0.478152,0.388446,0.112738,...,0.666770,0.185681,0.048184,0.633803,0.612884,0.771142,0.651123,0.865815,0.274747,0.531250
436,0.757895,0.892137,0.228933,0.582524,0.473498,0.996815,0.709085,0.199270,0.734372,0.000000,...,0.323283,0.220923,0.041159,0.718310,0.865563,0.895877,0.436960,0.706070,0.204040,0.364583
211,0.652632,0.704637,0.655899,0.893204,0.270671,0.925159,0.606827,0.606325,0.310117,0.023426,...,0.626360,0.194009,0.041898,0.570423,0.751461,0.903422,0.594128,0.843450,0.432323,0.510417
192,0.621053,0.584677,0.685393,0.388350,0.402827,0.598726,0.524448,0.493338,0.388446,0.071742,...,0.545539,0.256850,0.044298,0.637324,0.884681,1.000000,0.656304,0.699681,0.345455,0.562500
450,0.494737,0.497984,0.470506,0.524272,0.725795,0.821656,0.510112,0.116077,0.833450,0.135432,...,0.646565,0.144896,0.043059,0.806338,0.896736,0.850323,0.578584,0.779553,0.373737,0.781250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.736842,0.204637,0.811096,0.223301,1.000000,0.447452,0.108579,0.076960,0.878788,0.381406,...,0.484924,0.304050,0.048511,0.647887,0.612884,0.660744,0.787565,0.627263,0.356902,0.715278
106,0.852632,0.419355,0.678371,0.155340,0.658657,0.496815,0.247451,0.265543,0.655052,0.434846,...,0.545539,0.247601,0.049039,0.697183,0.676571,0.694407,0.658031,0.837061,0.383838,0.770833
270,0.557895,0.618952,0.766854,0.660194,0.166784,0.914013,0.590927,0.781879,0.175160,0.111274,...,0.663876,0.180807,0.041659,0.714789,0.849001,0.256860,0.571675,0.690096,0.309091,0.604167
435,0.757895,0.892137,0.228933,0.582524,0.473498,0.996815,0.709085,0.199270,0.734372,0.000000,...,0.495026,0.198373,0.043042,0.718310,0.865563,0.895877,0.069085,0.057508,0.115152,0.177083


In [215]:
# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1, solver='svd')  # Regularization strength

# Train the model
ridge_model.fit(X_train.to_numpy(), y_train)

# Predict on test set
y_pred_rr = ridge_model.predict(X_test.to_numpy())
y_pred_rr = scaler_y.inverse_transform(y_pred_rr)
y_pred_rr_df = pd.DataFrame(y_pred_rr, columns=output_cols)
X_test_inverted = scaler_x.inverse_transform(X_test)
X_test_inverted = pd.DataFrame(X_test_inverted, columns=input_cols)
inverted_pred_rr_df = pd.concat([X_test_inverted.reset_index(drop=True), y_pred_rr_df.reset_index(drop=True)], axis=1)
inverted_pred_rr_df

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2.055,13.265,39.840,0.460,23.195,1.800,6.451669,1.718814,0.590599,13.195,...,21.434619,1.032012,1.138098,74.659137,0.522043,0.061797,9.853076,642.742763,-3.596433,0.200082
1,2.070,15.505,37.615,0.510,20.935,2.930,7.491014,1.800052,0.566749,10.155,...,21.466702,1.185195,1.161662,74.394385,0.579230,0.066179,8.777381,623.463701,-3.224005,0.238611
2,2.070,12.310,42.130,0.445,22.910,1.505,5.949930,1.838900,0.551500,11.580,...,21.890556,1.258247,1.222987,73.836141,0.525850,0.067656,8.079174,604.315147,-3.490952,0.588601
3,2.055,15.165,37.190,0.560,21.310,0.000,7.378908,1.746586,0.583696,10.305,...,21.373712,1.134551,1.070287,74.721941,0.496186,0.063651,9.846618,614.190810,-3.309884,-0.082742
4,1.960,15.410,35.165,0.570,19.755,3.005,7.861458,1.780050,0.573375,14.105,...,21.877754,1.062699,1.000229,74.388752,0.521484,0.063069,10.250483,618.134206,-3.337657,0.038351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2.070,15.200,36.470,0.550,21.290,2.980,7.350000,1.710000,0.590000,10.320,...,21.581631,1.194465,1.217031,74.289400,0.478729,0.058827,8.836824,619.819149,-3.323341,0.108005
113,1.995,14.075,38.155,0.610,20.610,2.620,7.055126,1.851237,0.551703,10.870,...,19.880029,1.290462,1.470838,75.576145,0.441067,0.051607,4.881870,655.927740,-3.344603,0.045574
114,2.055,12.525,40.580,0.450,22.610,1.870,6.100760,1.795170,0.565000,11.925,...,21.891389,1.257396,1.241540,73.853903,0.500607,0.068438,7.993305,606.324216,-3.149074,0.415379
115,2.030,14.775,36.970,0.595,21.000,2.985,7.281335,1.761539,0.579661,10.035,...,21.062814,1.028139,0.980713,75.244888,0.530243,0.061193,8.954138,644.120816,-2.653998,0.430996


In [216]:
inverted_y_actual = scaler_y.inverse_transform(y_test)
inverted_y_actual = pd.DataFrame(inverted_y_actual, columns=output_cols)
inverted_y_pred_rr = y_pred_rr_df[output_cols]
inverted_y_pred_rr

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1378.638769,21.434619,1.032012,1.138098,74.659137,0.522043,0.061797,9.853076,642.742763,-3.596433,0.200082
1,1373.096868,21.466702,1.185195,1.161662,74.394385,0.579230,0.066179,8.777381,623.463701,-3.224005,0.238611
2,1377.305611,21.890556,1.258247,1.222987,73.836141,0.525850,0.067656,8.079174,604.315147,-3.490952,0.588601
3,1383.747788,21.373712,1.134551,1.070287,74.721941,0.496186,0.063651,9.846618,614.190810,-3.309884,-0.082742
4,1379.140036,21.877754,1.062699,1.000229,74.388752,0.521484,0.063069,10.250483,618.134206,-3.337657,0.038351
...,...,...,...,...,...,...,...,...,...,...,...
112,1384.442789,21.581631,1.194465,1.217031,74.289400,0.478729,0.058827,8.836824,619.819149,-3.323341,0.108005
113,1399.534726,19.880029,1.290462,1.470838,75.576145,0.441067,0.051607,4.881870,655.927740,-3.344603,0.045574
114,1390.519631,21.891389,1.257396,1.241540,73.853903,0.500607,0.068438,7.993305,606.324216,-3.149074,0.415379
115,1356.460567,21.062814,1.028139,0.980713,75.244888,0.530243,0.061193,8.954138,644.120816,-2.653998,0.430996


In [217]:
inverted_y_actual

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1377.000000,21.445000,1.030000,1.125000,74.661500,0.528500,0.064000,10.174000,640.000000,-4.000000,0.00
1,1374.500000,21.173333,1.188333,1.158333,74.656833,0.614833,0.066667,9.256667,660.000000,-2.700000,0.68
2,1346.400000,21.084000,1.326000,1.172000,74.524200,0.569800,0.070000,8.080000,611.000000,-3.800000,0.54
3,1386.666667,21.540000,1.130000,1.086667,74.542000,0.508667,0.064000,10.108000,592.000000,-3.100000,0.00
4,1373.000000,21.647500,1.065000,0.972500,74.673000,0.486000,0.062500,10.165000,635.285714,-3.357143,0.37
...,...,...,...,...,...,...,...,...,...,...,...
112,1393.666667,21.778333,1.223333,1.236667,74.087500,0.445833,0.055000,8.595000,638.250000,-3.125000,0.00
113,1394.875000,19.708750,1.256250,1.436250,75.828000,0.439500,0.050000,4.990000,653.000000,-1.700000,0.00
114,1389.000000,21.947500,1.262500,1.225000,73.847250,0.488750,0.065000,7.810000,602.000000,-2.400000,0.48
115,1394.875000,19.708750,1.256250,1.436250,75.828000,0.439500,0.050000,4.990000,612.000000,-2.000000,0.65


In [218]:
models = {
    'RR': inverted_y_pred_rr,

}
mae_dict = {'Column': output_cols}
# Iterate through each model and calculate MAE for each column
for model_name, inverted_y_pred in models.items():
    mae_dict[model_name] = []  # Add a column for each model
    for col in output_cols:
        # Extract actual and predicted values
        y_actual_col = inverted_y_actual[col]
        y_pred_col = inverted_y_pred[col]

        # Calculate MAE and store in the dictionary
        mae_col = mean_absolute_error(y_actual_col, y_pred_col)
        mae_dict[model_name].append(mae_col)

# Convert the dictionary to a DataFrame
metrics_df = pd.DataFrame(mae_dict)

# Display the DataFrame
display(metrics_df)

,Column,RR
0,furnace_temp,9.292703
1,ni_met,0.256619
2,c_met,0.032770
3,si_met,0.024445
4,fe_met,0.218259
5,s_met,0.032141
6,ni_slag,0.003389
7,fe_slag,0.268211
8,t_kalsin,17.689237
9,pic_161,0.460935


In [219]:
target_value = y_train.iloc[0, :].to_numpy()
print(target_value)

def objective_function(x_scaled):
    # untuk  prediksi 1 fitur output saja dengan 1 sample saja pada 30 fitur input
    x_scaled=x_scaled.reshape(1,-1)
    y_pred = ridge_model.predict(x_scaled)[0]
    # print(ridge_model.predict(x_scaled))
    return (y_pred[0] - target_value[0]) **2

# def objective_function(x_scaled):
#     # untuk  prediksi 1 fitur output saja dengan 1 sample saja pada 30 fitur input
#     x_scaled=x_scaled.reshape(1,-1)
#     y_pred = ridge_model.predict(x_scaled)[0]
#     # print(ridge_model.predict(x_scaled))
#     return y_pred[0]


[0.         0.6958021  0.65342163 0.2543021  0.33788151 0.69251578
 0.90909091 0.59606481 0.3778626  0.26027397 0.44230769]


In [220]:
# untuk  1 sample saja pada 30 fitur input
x0=X_train.iloc[0,:].to_numpy().flatten()


result = minimize(objective_function, x0, method='SLSQP')

# Extract the optimized inputs and predicted output
optimized_x = result.x
print("optimized_x",optimized_x)
optimized_y = ridge_model.predict(optimized_x.reshape(1,-1))[0]

print("optimized_y:",optimized_y)


optimized_x [0.8508653  0.60274586 0.72513004 0.32741212 0.38554349 0.57290849
 0.32846023 0.34530903 0.30686512 0.09750285 0.63610841 0.85119188
 0.78444186 0.2858052  0.83773885 0.18625815 0.99192115 0.55738838
 0.87400622 1.00820588 1.01490074 0.99426359 0.6477027  0.16865671
 0.06464716 0.6896285  0.60491094 0.78457038 0.5943579  0.92673305
 0.22987801 0.52444799]
optimized_y: [-3.91623406e-09  8.53399395e-01  4.06156900e-01  2.30350693e-01
  1.84039825e-01  7.69933709e-01  8.92208818e-01  6.52511932e-01
  3.43348296e-01  3.05518329e-01  6.21259670e-01]


In [221]:
a=pd.DataFrame(optimized_x).T
a.columns=input_cols
a=pd.DataFrame(scaler_x.inverse_transform(a), columns=input_cols)
print(a.shape)
b=pd.DataFrame(optimized_y).T
b=pd.DataFrame(scaler_y.inverse_transform(b),columns=output_cols)
print(b.shape)

(1, 32)
(1, 11)


In [222]:
b

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1346.4,21.784783,1.213989,1.15321,74.010061,0.586971,0.069629,8.372622,581.957253,-3.369716,0.64611


In [223]:
df[output_cols]

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,627.0,-3.0,0.71
1,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,607.0,-2.7,0.71
2,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,614.0,-3.1,0.71
3,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,620.0,-0.9,0.90
4,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,610.0,-3.0,0.90
...,...,...,...,...,...,...,...,...,...,...,...
579,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,602.0,-4.4,0.00
580,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,615.0,-3.8,0.00
581,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,622.0,-3.5,0.00
582,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,615.0,-3.2,0.00


In [224]:
a

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.119161,13.959619,40.172926,0.458617,21.11772,1.798933,6.389342,1.782525,0.5247,10.245944,...,5005.630134,7.355739,60.967904,6.288545,128.895286,2236.026918,966.133226,822.067446,485.789612,269.347007


In [225]:
min_values = scaler_y.data_min_
max_values = scaler_y.data_max_

print(min_values.shape)
print(max_values.shape)
konstrain=[]
bounds=[ (i,j) for i, j in zip(min_values, max_values)]
print(bounds)

T_furnace=1300
T_furnace_norm= (T_furnace-bounds[0][0])/(bounds[0][1]-bounds[1][0])
print("T_furnace_norm",T_furnace_norm)

ni_met=17
ni_met_norm= (ni_met-bounds[1][0])/(bounds[1][1]-bounds[1][0])
print("Ni_norm",ni_met_norm)

C_met_low=1.0
C_met_low_norm= (C_met_low-bounds[2][0])/(bounds[2][1]-bounds[2][0])
print("C_met_low_norm",C_met_low_norm)

C_met_high=2.0
C_met_high_norm= (C_met_high-bounds[2][0])/(bounds[2][1]-bounds[2][0])
print("C_met_high_norm",C_met_high_norm)

Si_met_low=1
Si_met_low_norm= (Si_met_low-bounds[3][0])/(bounds[3][1]-bounds[3][0])
print("Si_met_low_norm",Si_met_low_norm)

Si_met_high=2
Si_met_high_norm= (Si_met_high-bounds[3][0])/(bounds[3][1]-bounds[3][0])
print("Si_met_high_norm",Si_met_high_norm)


fe_met=76
fe_met_norm= (fe_met-bounds[4][0])/(bounds[4][1]-bounds[4][0])
print("fe_met_norm",fe_met_norm)

s_met=0.4
s_met_norm= (s_met-bounds[5][0])/(bounds[5][1]-bounds[5][0])
print("s_met_norm",s_met_norm)

ni_slag=0.07
ni_slag_norm= (ni_slag-bounds[6][0])/(bounds[6][1]-bounds[6][0])
print("ni_slag_norm",ni_slag_norm)

fe_slag=10
fe_slag_norm= (fe_slag-bounds[7][0])/(bounds[7][1]-bounds[7][0])
print("fe_slag_norm",fe_slag_norm)

T_kalsin=600
T_kalsin_norm= (T_kalsin-bounds[8][0])/(bounds[8][1]-bounds[8][0])
print("T_kalsin_norm",T_kalsin_norm)

pic_161=-2
pic_161_norm= (pic_161-bounds[9][0])/(bounds[9][1]-bounds[9][0])
print("pic_161_norm",pic_161_norm)

loi_kalsin_low=0
loi_kalsin_low_norm= (loi_kalsin_low-bounds[10][0])/(bounds[10][1]-bounds[10][0])
print("loi_kalsin_low_norm",loi_kalsin_low_norm)

loi_kalsin_high=1
loi_kalsin_high_norm= (loi_kalsin_high-bounds[10][0])/(bounds[10][1]-bounds[10][0])
print("loi_kalsin_high_norm",loi_kalsin_high_norm)




(11,)
(11,)
[(1346.4, 1405.5), (17.990000000000002, 22.436666666666664), (1.03, 1.4829999999999999), (0.9725, 1.7570000000000001), (73.395, 76.737), (0.4161999999999999, 0.6379999999999999), (0.05, 0.07200000000000001), (4.99, 10.174000000000001), (492.0, 754.0), (-5.6, 1.7), (0.0, 1.04)]
T_furnace_norm -0.03344120042378079
Ni_norm -0.22263868065967085
C_met_low_norm -0.06622516556291398
C_met_high_norm 2.1412803532008837
Si_met_low_norm 0.0350541746335245
Si_met_high_norm 1.3097514340344165
fe_met_norm 0.7794733692399776
s_met_norm -0.07303877366997241
ni_slag_norm 0.9090909090909091
fe_slag_norm 0.9664351851851849
T_kalsin_norm 0.4122137404580153
pic_161_norm 0.4931506849315068
loi_kalsin_low_norm 0.0
loi_kalsin_high_norm 0.9615384615384615


In [226]:
a

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.119161,13.959619,40.172926,0.458617,21.11772,1.798933,6.389342,1.782525,0.5247,10.245944,...,5005.630134,7.355739,60.967904,6.288545,128.895286,2236.026918,966.133226,822.067446,485.789612,269.347007


In [227]:
# min_values = scaler_y.data_min_
# max_values = scaler_y.data_max_

# print(min_values.shape)
# print(max_values.shape)
# konstrain=[]
# bounds=[ (i,j) for i, j in zip(min_values, max_values)]

# ni_in=0
# ni_in_norm= (ni_in-bounds[11][0])/(bounds[11][1]-bounds[11][0])
# print("ni_in_norm",ni_in_norm)

In [228]:
df[input_cols].describe()

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,...,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,2.044932,13.982295,38.377509,0.544212,21.471045,2.310120,6.857324,1.793400,0.570179,11.063339,...,4710.212724,7.564634,51.695658,6.260582,131.569315,2093.403630,971.607877,770.391267,559.991438,281.118151
std,0.070222,1.023608,1.720118,0.110906,1.295725,0.664713,0.588834,0.114457,0.033447,1.130211,...,620.920506,0.625678,26.693039,0.506534,20.716518,309.703286,95.906588,35.526280,43.858463,16.792316
min,1.715000,10.970000,35.010000,0.290000,18.390000,0.000000,5.500000,1.555143,0.467150,9.580000,...,1800.000000,5.483684,23.298217,4.330000,79.220000,1369.720000,622.000000,532.000000,372.000000,219.000000
25%,2.005000,13.210000,36.780000,0.460000,20.650000,1.760000,6.410000,1.723282,0.550000,10.320000,...,4399.927885,7.225611,47.481606,6.110000,127.350000,2023.140000,929.000000,753.000000,534.220588,269.078947
50%,2.040000,14.030000,38.130000,0.560000,21.470000,2.520000,6.896463,1.776755,0.573967,10.850000,...,4800.000000,7.483200,50.056917,6.290000,134.780000,2136.470000,958.342105,774.000000,561.000000,283.000000
75%,2.090000,14.870000,39.926250,0.620000,22.252500,2.905000,7.350000,1.840000,0.590000,11.520000,...,5200.000000,7.833375,52.241378,6.430000,148.940000,2308.630000,996.000000,793.236842,585.000000,294.000000
max,2.190000,15.930000,42.130000,0.805000,25.465000,3.140000,8.207611,2.213630,0.654692,16.410000,...,6749.230769,16.583478,605.994944,7.170000,161.340000,2473.900000,1201.000000,845.000000,867.000000,315.000000


In [229]:
# constraints = [
#     {'type': 'ineq', 'fun': lambda x: x[0] - T_furnace_norm},  # T_furnace > 1300

#     {'type': 'ineq', 'fun': lambda x: x[1] - ni_met_norm},  # Ni_met > 17


#     #{'type': 'ineq', 'fun': lambda x: x[2] - C_met_low_norm},  # C_met > 1
#     # {'type': 'ineq', 'fun': lambda x: C_met_high_norm- x[2]}, # C_met < 2
    

#     {'type': 'ineq', 'fun': lambda x: x[3] - Si_met_low_norm},  # Si_met > 1
#     {'type': 'ineq', 'fun': lambda x: Si_met_high_norm- x[3]},  # Si_met < 2
    

#     # {'type': 'ineq', 'fun': lambda x: x[4] - fe_met_norm},  # Fe_met > 76


#     {'type': 'ineq', 'fun': lambda x: s_met_norm - x[5] },  # S_met < 0.4


#     # {'type': 'ineq', 'fun': lambda x: x[6] - ni_slag_norm},  # Ni_slag > 0.07
#     # {'type': 'ineq', 'fun': lambda x: x[7] - fe_slag_norm},  # Fe_slag > 10


#     {'type': 'ineq', 'fun': lambda x: x[8] - T_kalsin_norm},    # T_kalsin > 600

#     {'type': 'ineq', 'fun': lambda x: x[9] - pic_161_norm},  # pic_161 > -2

#     {'type': 'ineq', 'fun': lambda x: loi_kalsin_norm - x[10]}, # loi_kalsin < 1

# ]

constraints = [
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][0] - T_furnace_norm},  # T_furnace > 1300
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][1] - ni_met_norm},    # Ni_met > 17
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][3] - Si_met_low_norm},  # Si_met > 1
    {'type': 'ineq', 'fun': lambda x: Si_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][3]},  # Si_met < 2
    {'type': 'ineq', 'fun': lambda x: s_met_norm - ridge_model.predict(x.reshape(1, -1))[0][5]},  # S_met < 0.4
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][8] - T_kalsin_norm},    # T_kalsin > 600
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][9] - pic_161_norm},  # pic_161 > -2
    {'type': 'ineq', 'fun': lambda x: loi_kalsin_high - ridge_model.predict(x.reshape(1, -1))[0][10]},  # loi_kalsin < 1
    {'type': 'ineq', 'fun': lambda x: ridge_model.predict(x.reshape(1, -1))[0][2] - loi_kalsin_low },  # C_met > 0
]


x0 = X_train.iloc[0,:].to_numpy().flatten()
print(x0.shape)
# Jalankan optimisasi dengan constraints
result = minimize(objective_function, x0, method='SLSQP', constraints=constraints)

optimized_x = result.x
optimized_y = ridge_model.predict(optimized_x.reshape(1,-1))[0]

print("Optimized x:", optimized_x)
print("Optimized y:", optimized_y)

(32,)
Optimized x: [ 0.83255107  0.43891618  0.70794523  0.34129731  0.47861048  0.50007827
  0.4845616   0.21671179  0.09334217  0.15543852  0.49559161  0.99118266
  0.78918231  0.27941357  0.86600158  0.04240097  0.92144721  0.5539846
  1.17598118  0.84329186  1.04535892  0.87794028  0.59085179  0.01110443
 -0.13232683  0.8283765   0.51596575  0.70850093  0.60522598  0.97692762
  0.64814665  0.37482808]
Optimized y: [-4.60033289e-09  1.15214544e+00  3.61397702e-01  4.33309151e-02
 -9.14996690e-02 -8.27834848e-02  7.02204267e-01  5.57926592e-01
  5.68897108e-01  4.93150695e-01  9.90334654e-01]


In [230]:
a=pd.DataFrame(optimized_x).T
a.columns=input_cols
a=pd.DataFrame(scaler_x.inverse_transform(a), columns=input_cols)
print(a.shape)
b=pd.DataFrame(optimized_y).T
b=pd.DataFrame(scaler_y.inverse_transform(b),columns=output_cols)
print(b.shape)

(1, 32)
(1, 11)


In [231]:
a

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.110462,13.147024,40.05057,0.465768,21.776169,1.570246,6.812004,1.697845,0.484656,10.641645,...,4724.261873,5.606941,-53.808195,6.682589,121.591107,2152.032558,972.425844,837.778344,692.832594,254.983495


In [232]:
b

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1346.4,23.113207,1.193713,1.006493,73.089208,0.397839,0.065448,7.882291,641.051042,-2.0,1.029948


In [233]:
df[input_cols]

,ni_in,fe_in,sio2_in,cao_in,mgo_in,al2o3_in,fe_ni,s_m,bc,loi_in,...,total_coal,a_f_ratio,kg_tco,reductor_ratio,reductor_consume,charge_kiln,t_tic162,t_tic163,t_tic166,t_tic172
0,2.05,12.770,37.580,0.45,20.30,1.370,6.230000,1.860000,0.550000,11.170,...,4500.0,7.898667,51.658822,6.16,127.09,2064.34,1200.0,757.0,532.0,270.0
1,2.00,10.970,36.390,0.36,19.54,1.130,5.500000,1.860000,0.540000,12.980,...,4600.0,7.623913,50.179993,6.29,127.09,2064.34,1200.0,756.0,504.0,270.0
2,2.00,10.970,36.390,0.36,19.54,1.130,5.500000,1.860000,0.540000,12.980,...,4600.0,7.560000,51.099756,6.29,127.09,2064.34,1200.0,760.0,530.0,276.0
3,2.00,10.970,36.390,0.36,19.54,1.130,5.500000,1.860000,0.540000,12.980,...,4300.0,7.950698,49.710983,6.29,127.09,2064.34,1200.0,762.0,714.0,283.0
4,2.00,10.970,36.390,0.36,19.54,1.130,5.500000,1.860000,0.540000,12.980,...,4300.0,7.950698,49.499252,6.29,127.09,2064.34,1200.0,747.0,535.0,280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,2.04,13.465,37.045,0.53,22.25,2.335,6.610728,1.666057,0.610755,10.945,...,3800.0,8.254737,54.841969,4.33,79.22,1831.57,799.0,753.0,550.0,262.0
580,2.04,13.465,37.045,0.53,22.25,2.335,6.610728,1.666057,0.610755,10.945,...,3800.0,8.125263,55.232558,4.33,79.22,1831.57,804.0,749.0,540.0,259.0
581,2.04,13.465,37.045,0.53,22.25,2.335,6.610728,1.666057,0.610755,10.945,...,3700.0,8.317297,52.940335,4.33,79.22,1831.57,793.0,765.0,608.0,275.0
582,2.04,13.465,37.045,0.53,22.25,2.335,6.610728,1.666057,0.610755,10.945,...,3800.0,7.935789,53.109713,4.33,79.22,1831.57,841.0,787.0,602.0,271.0


In [234]:
df[output_cols]

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,627.0,-3.0,0.71
1,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,607.0,-2.7,0.71
2,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,614.0,-3.1,0.71
3,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,620.0,-0.9,0.90
4,1399.200,20.80000,1.29400,1.28400,74.8144,0.5044,0.070,7.220000,610.0,-3.0,0.90
...,...,...,...,...,...,...,...,...,...,...,...
579,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,602.0,-4.4,0.00
580,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,615.0,-3.8,0.00
581,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,622.0,-3.5,0.00
582,1394.875,19.70875,1.25625,1.43625,75.8280,0.4395,0.050,4.990000,615.0,-3.2,0.00


In [235]:
df[output_cols].describe()

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,1385.873288,21.485707,1.191549,1.188949,74.397347,0.501617,0.063729,8.682075,615.315068,-3.296062,0.270642
std,16.083578,0.578239,0.084644,0.100770,0.528813,0.052588,0.005112,1.194774,26.863995,0.726518,0.300675
min,1346.400000,17.990000,1.030000,0.972500,73.395000,0.416200,0.050000,4.990000,492.000000,-5.600000,0.000000
25%,1377.000000,21.243333,1.145000,1.131667,73.972000,0.462167,0.060000,8.020000,600.000000,-3.800000,0.000000
50%,1389.000000,21.540000,1.188333,1.180000,74.524200,0.487625,0.064000,8.723333,613.000000,-3.300000,0.150000
75%,1399.200000,21.860000,1.256250,1.238750,74.673000,0.528500,0.066667,9.784000,627.000000,-2.900000,0.530000
max,1405.500000,22.436667,1.483000,1.757000,76.737000,0.638000,0.072000,10.174000,754.000000,1.700000,1.040000
